# Recursive Feature Elimination of MRI Features
In this notebook MRI based features of Cerebral Blood Flow(CBF), arterial Blood Volume(aBV), Fractional Anisotropy(FA), Mean Diffusivity(MD), functional MRI(fMRI), spectroscopy are used to predict Parkinson's Disease Mild Cognitive Impairment (PD-MCI).
Most informative features are tried to be found as biomarkers indicating PD-MCI status. 

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pylab 
import scipy.stats as stats
import time
import sklearn.ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from pandas import ExcelWriter
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.svm import LinearSVC

from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.linear_model import LogisticRegression
import collections

import re

import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second

## Reading Excel Files
Files containing each modalities are imported as DataFrames. MR spectroscopic data are imported from four different files including four different metabolite ratio values of Glutamine (Glu), N-acetyl aspartate (Naa), Myo-inositol (Ins) and Choline divided by Creatine.

In [ ]:
cd C:\Users\DELL003\Desktop\PD_Documents

In [ ]:
FA = pd.read_excel('DTI.xlsx', sheet_name = 'FA', index_col='ID')
FA.shape

In [ ]:
MD = pd.read_excel('DTI.xlsx', sheet_name = 'MD', index_col='ID')
MD.shape

In [ ]:
aBV = pd.read_excel('aBV.xlsx', sheet_name = 'Mean', index_col='ID')
aBV.shape

In [ ]:
CBF = pd.read_excel('CBF.xlsx', sheet_name = 'Mean', index_col='ID')
CBF.shape

In [ ]:
fmri = pd.read_excel('fmri.xlsx', sheet_name = 'Sheet1', index_col='ID')
fmri.shape

In [ ]:
group = pd.read_excel('group.xlsx', sheet_name = 'Sheet1', index_col='ID')
group.shape

In [ ]:
glu = pd.read_excel('spectroscopy.xlsx', sheet_name = 'Glu', index_col='ID')
naa = pd.read_excel('spectroscopy.xlsx', sheet_name = 'Naa', index_col='ID')
ins = pd.read_excel('spectroscopy.xlsx', sheet_name = 'Ins', index_col='ID')
cho = pd.read_excel('spectroscopy.xlsx', sheet_name = 'Cho', index_col='ID')

Getting only MRI features

In [ ]:
glu = glu.loc[:, glu.columns.str.contains('7Networks', regex=True)]
naa = naa.loc[:, naa.columns.str.contains('7Networks', regex=True)]
ins = ins.loc[:, ins.columns.str.contains('7Networks', regex=True)]
cho = cho.loc[:, cho.columns.str.contains('7Networks', regex=True)]

Adding suffixes to each feature types.

In [ ]:
glu.columns = glu.columns + '_glu'
naa.columns = naa.columns + '_naa'
ins.columns = ins.columns + '_ins'
cho.columns = cho.columns + '_cho'
CBF.columns = pd.Series([str(col) for col in CBF.columns]) + '_CBF'
aBV.columns = pd.Series([str(col) for col in aBV.columns]) + '_aBV'
FA.columns = FA.columns + '_FA'
MD.columns = MD.columns + '_MD'
fmri.columns = fmri.columns + '_fmri'

Equaling index names in order to merge the DataFrames.

In [ ]:
glu_index = pd.Series(glu.index)
naa_index = pd.Series(naa.index)
ins_index = pd.Series(ins.index)
cho_index = pd.Series(cho.index)
CBF_index = pd.Series(CBF.index)
aBV_index = pd.Series(aBV.index)

In [ ]:
string_list = ['_Glu+Gln_Conc_T2_overlaid_zeros.nii.gz', 'reg_', 'NAA+NAAG_Conc_T2_overlaid_zeros.nii.gz',
               '_Ins_Conc_T2_overlaid_zeros.nii.gz', '_GPC+PCh_Conc_T2_overlaid_zeros.nii.gz',
               '_brain.nii_normmi_flirt_CBF_05_masked.nii', '_T1_07', '_T1_05',
               '_brain.nii_normmi_flirt_CBF_03_masked.nii',
               '_brain.nii_normmi_flirt_CBF_07_masked.nii', '_']

for string in string_list:
    glu_index = glu_index.apply(lambda x: x.replace(string, ''))
    naa_index = naa_index.apply(lambda x: x.replace(string, ''))
    ins_index = ins_index.apply(lambda x: x.replace(string, ''))
    cho_index = cho_index.apply(lambda x: x.replace(string, ''))
    CBF_index = CBF_index.apply(lambda x: x.replace(string, ''))
    aBV_index = aBV_index.apply(lambda x: x.replace(string, ''))   

In [ ]:
glu.set_index(glu_index, inplace=True)
naa.set_index(naa_index, inplace=True)
ins.set_index(ins_index, inplace=True)
cho.set_index(cho_index, inplace=True)
CBF.set_index(CBF_index, inplace=True)
aBV.set_index(aBV_index, inplace=True)

Merging the DataFrames.

In [ ]:
data = group
df_list = [fmri, CBF, aBV, FA, MD, glu, cho, ins, naa]

for df in df_list:
    data = pd.merge(data, df, on='ID', how='left')
    
data.shape

Dropping some features and samples containing high amount of missing values.

In [ ]:
#Dropping NA
data = data.dropna(axis = 1, thresh = round((0.5)*data.shape[0])) #removing features existing in less than half of the samples 
data = data.dropna(axis = 0, thresh = round((0.5)*data.shape[1])) #removing samples having features less than half of the total features 
print(data.shape)

Examining the merged DataFrame

In [ ]:
pd.set_option('display.max_columns', None) 
pd.options.display.max_rows = 100
data

Examining the number of missing values in each feature.

In [ ]:
#Checking missing values in features
pd.options.display.max_rows = 1000
data.isna().sum()

In [ ]:
data.max().max()

Filling the missing values by a constant that is not contained in the data.

In [ ]:
data = data.fillna(9999)

Examining the number of paricipants in each group.

In [ ]:
print('Number of Healthy Controls:', (data.group == 0).sum())
print('Number of Cognitively Normal Parkinson Disease Patients:', (data.group == 1).sum())
print('Number of Mild Cognitive Parkinson Disease Patient:', (data.group == 2).sum())

Visualizing the pearson correlations between each features. 

In [ ]:
corr_matrix = data.corr()

plt.figure(figsize = (12,8))
sns.heatmap(corr_matrix)

plt.show()

Decomposing the modalities from the merged data frames.

In [ ]:
fmri_reduced = data.loc[:, data.columns.str.contains('_fmri', regex=True)]
aBV_reduced = data.loc[:, data.columns.str.contains('_aBV', regex=True)]
FA_reduced = data.loc[:, data.columns.str.contains('_FA', regex=True)]
MD_reduced = data.loc[:, data.columns.str.contains('_MD', regex=True)]
CBF_reduced = data.loc[:, data.columns.str.contains('_CBF', regex=True)]
spectroscopy_reduced = data.loc[:, data.columns.str.contains('_glu|_cho|_ins|_naa', regex=True)]

Examining the explained variances of each modalities using Principal Component Analysis (PCA). 

In [ ]:
n_pca_components = 5

print('Total explined variance with PCA of CBF features is {}'.format(PCA(n_components=n_pca_components).fit((CBF_reduced)).explained_variance_ratio_.sum()))
print('Total explined variance with PCA of aBV features is {}'.format(PCA(n_components=n_pca_components).fit((aBV_reduced)).explained_variance_ratio_.sum()))
print('Total explined variance with PCA of FA features is {}'.format(PCA(n_components=n_pca_components).fit((FA_reduced)).explained_variance_ratio_.sum()))
print('Total explined variance with PCA of MD features is {}'.format(PCA(n_components=n_pca_components).fit((MD_reduced)).explained_variance_ratio_.sum()))
print('Total explined variance with PCA of fmri features is {}'.format(PCA(n_components=n_pca_components).fit((fmri_reduced)).explained_variance_ratio_.sum()))
print('Total explined variance with PCA of spectroscopy features is {}'.format(PCA(n_components=n_pca_components).fit((spectroscopy_reduced)).explained_variance_ratio_.sum()))

Setting the number of features that will be selected as the most important in each cycle.

In [ ]:
#Change number of selected features
number_of_selected_features = 1

Initializing Recursive Feature Elimination (RFE) objects with RandomForestClassifier and ExtraTreesClassifier.

In [ ]:
#Feature Selection Algorithms
fs1 = RFE(RandomForestClassifier(n_estimators=100), n_features_to_select=number_of_selected_features, verbose=2)
fs2 = RFE(ExtraTreesClassifier(n_estimators=100, bootstrap=True), n_features_to_select=number_of_selected_features, verbose=2)

#Add more feature selection options

Defining the functions that will select each modality in pipelines. 

In [ ]:
get_fmri = FunctionTransformer(lambda x: x[list(fmri_reduced.columns)], validate=False)
get_aBV = FunctionTransformer(lambda x: x[list(aBV_reduced.columns)], validate=False)
get_CBF = FunctionTransformer(lambda x: x[list(CBF_reduced.columns)], validate=False)
get_FA = FunctionTransformer(lambda x: x[list(FA_reduced.columns)], validate=False) 
get_MD = FunctionTransformer(lambda x: x[list(MD_reduced.columns)], validate=False)
get_spectroscopy = FunctionTransformer(lambda x: x[list(spectroscopy_reduced.columns)], validate=False)

Joining the preprocessing steps for each modality with FeatureUnion.

In [ ]:
# Create a FeatureUnion with nested pipeline: process_and_join_features
join_features = FeatureUnion(
            transformer_list = [
                ('aBV_features', Pipeline([
                    ('selector', get_aBV),
                    ('pca', PCA(n_components=n_pca_components))
                ])),
                ('FA_features', Pipeline([
                    ('selector', get_FA),
                    ('pca', PCA(n_components=n_pca_components))
                ])),
                 ('MD_features', Pipeline([
                    ('selector', get_MD),
                    ('pca', PCA(n_components=n_pca_components))
                ])),
                ('spectroscopy_features', Pipeline([
                    ('selector', get_spectroscopy),
                    ('pca', PCA(n_components=n_pca_components))
                ])),
                ('fmri_features', Pipeline([
                    ('selector', get_fmri)
                ])),
                ('CBF_features', Pipeline([
                    ('selector', get_CBF)
                ]))
                
             ]
        )


Defining the pipelines for feature selection.

In [ ]:
#Pipelines

pipe1 = Pipeline([('union', join_features),
                 ('fs', fs1)])

pipe2 = Pipeline([('union', join_features),
                 ('fs', fs2)])


#Add more pipelines



In [ ]:
pipelist = [pipe1, pipe2] 

Calculating the time required for the features selection.

In [ ]:
start = time.time()
for pipe in pipelist:
    pipe = pipe.fit((data.drop(labels=['group'], axis=1)), data['group'])

end = time.time()

In [ ]:
num_of_loops = 100
Estimated_Time = ((end - start)*num_of_loops)
print('Estimated Time:',Estimated_Time/60,'min','or',Estimated_Time/3600,'h')

Performing feature selection num_of_loops times with different train-test splits in each loop.

In [ ]:
start = time.time()
print('Program start to run at',time.localtime())
counter_collections=collections.Counter()


for pipe in pipelist:
    selectedfeatures=[]

    for i in range(num_of_loops):

        X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['group'], axis=1), data['group'], test_size=0.2)


        # run pipelines
        pipe = pipe.fit(X_train, y_train)
        selectedfeatures.append(tuple(pipe.steps[1][1].get_support(indices=True)))          #for RF-RFE




    counter=collections.Counter(selectedfeatures)
    counter_collections = counter_collections + counter
    
    print('Most Common Selected Features:',np.array(counter.most_common))    #change according to number of selected features
    
    
    

print('Total Selected Features in all methods:',np.array(counter_collections.most_common))

end = time.time()
print('Computation Time:',(end - start)/60,'min')

Defining names of the transformed features.

In [ ]:
pca_col_list = []
pca_modality_list = ['aBV', 'FA', 'MD', 'spectroscopy']
for modality in pca_modality_list:
    for i in range(1, n_pca_components+1, 1):
        pca_col_list.append(modality + '_pca' + str(i))
        
for i in fmri_reduced.columns:
    pca_col_list.append(i)
    
for i in CBF_reduced.columns:
    pca_col_list.append(i)

Inserting the indices of the transformed features that are mostly selected.  

In [ ]:
selected_feature_index_list = [10, 21, 43, 76, 24]

In [ ]:
print('Selected features: ', [pca_col_list[index] for index in selected_feature_index_list]) 

Defining the classifier for performing classification with selected features.

In [ ]:
clf = RandomForestClassifier(n_estimators=100)

In [ ]:
clf = ExtraTreesClassifier(n_estimators=100)

In [ ]:
clf = SVC()

In [ ]:
clf = LogisticRegression()

In [ ]:
clf = KNeighborsClassifier()

Performing classification num_of_loops times and calculating mean accuracy and confusion matrix.

In [ ]:
accuracy = 0
conf_matrix = 0

for i in range(num_of_loops):
    X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['group'], axis=1), data['group'], test_size=0.2, stratify=data['group'])
    
    X_train_transformed = join_features.fit_transform(X_train)
    X_train_selected = X_train_transformed[:,selected_feature_index_list]    

    X_test_transformed = join_features.transform(X_test)
    X_test_selected = X_test_transformed[:,selected_feature_index_list]

    clf.fit(X_train_selected, y_train)
    accuracy += clf.score(X_test_selected, y_test)
    conf_matrix += confusion_matrix(y_test, clf.predict(X_test_selected))

mean_accuracy = accuracy / num_of_loops
mean_conf_matrix = conf_matrix / num_of_loops

print('Mean accuracy: ', mean_accuracy)
print('Mean confusion matrix: ', mean_conf_matrix)
